# **In this lab, in addition to some fun Breaking Bad references, you will learn to:**

	•	Apply Singular Value Decomposition (SVD) using numpy to compress an image.
	•	Explore SVD's (TruncatedSVD) limitation for non-linear dataset, and compare with t-sne, using the concept of dimensionality reduction.
	•	Explore SVD's limitation for sudden and high contrast changes in images with sharp edges such as images of text.
	•	Using Discrete Cosine Transform: a solution to images with sharp edges and sudden high contrast changes.


# **Recap**
## 1. SVD:
Singular Value Decomposition (SVD) is a matrix factorization technique that breaks down a matrix A into three matrices: 
A = U . S . V^T , where:

U (Left Singular Vectors) – Captures important row patterns. 

S (Singular Values) – Represents importance (energy) of each component. 

Vᵀ (Right Singular Vectors) – Captures important column patterns.

SVD can be used in image compression (lossy), dimensionality reduction, etc.

##### **Limitations**: 
1. When data has non-linear relationship. 

2. When an image has sudden changes and sharp edges, like images of text.

## 2. t-sne:
t-SNE (t-Distributed Stochastic Neighbor Embedding) is a non-linear dimensionality reduction technique used to visualize high-dimensional data in 2D or 3D. It takes high dimensional data (eg., 100D) and maps it to 2D or 3D by keeping similar points together.

##### **Advantage**: Works amazing for data with non-linear relationship, hence it overcomes the first limitation of SVD.

## 3. DCT:
DCT (Discrete Cosine Transform) is a mathematical transformation used for image and signal compression by converting data into frequency components.

It breaks the data into frequency components (low & high frequencies). Then, it keeps important (low-frequency) parts and removes less important (high-frequency) parts.

Difference between SVD and DCT: DCT works in the frequency domain (An image in the frequency domain means that it represents the variation of pixel intensities rather than the pixels themselves) while SVD works in the matrix domain.

##### **Advantage**: Works good for images with sudden changes and sharp edges, like images of text. Hence, it overcomes the second limitation of SVD.

# References and Materials:

**Breaking Bad (A must watch!)**: https://www.imdb.com/title/tt0903747/

**Singular Value Decomposition**: https://www.geeksforgeeks.org/singular-value-decomposition-svd/

**t-sne**: https://www.geeksforgeeks.org/ml-t-distributed-stochastic-neighbor-embedding-t-sne-algorithm/

**Discrete Cosine Transform**: https://www.geeksforgeeks.org/discrete-cosine-transform-algorithm-program/

# Before starting with the lab... 
ensure you have the required libraries installed. Run:

**pip install opencv-python numpy matplotlib scikit-learn scipy**

# Step 1: Load an Image in Grayscale

In [ ]:
import cv2
import matplotlib.pyplot as plt

def load_image(image_path):
    """Load image in grayscale"""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return img

image_path = "face.png"
img = load_image(image_path)

# Display the original image
plt.imshow(img, cmap="gray")
plt.title("Original Face Image (Walter White aka Heisenberg)")
plt.show()

: 

# Step 2: Write SVD using numpy

Complete the TODOs below

In [5]:
import numpy as np

def compress_image_svd(img, k):
    """Compress an image using SVD by keeping only k singular values"""
    # Step 1: Compute full SVD using numpy's SVD
    # U: Left Singular Vectors. S: Singular Values. Vt: Right Singular Vectors Transposed. 
    # Apply svd from linalg in numpy. 
    # Set full_matrices argument to False (makes the decomposition faster and memory-efficient by keeping only the necessary components).
    U, S, Vt = np.linalg.svd(img, full_matrices=False)
    # Step 2: Keep only top-k singular values
    S_k = np.zeros((k, k))  # Create a k x k Sigma matrix
    np.fill_diagonal(S_k, S[:k])  # Fill diagonal with top-k singular values
    U_k = U[:, :k]  # Take first k columns of U
    Vt_k = Vt[:k, :]  # Take first k rows of Vt

    # Step 3: Reconstruct the image using U dot S dot V^T (use dot function in numpy)
    compressed_img = np.dot(U_k, np.dot(S_k, Vt_k))

    return np.clip(compressed_img, 0, 255).astype(np.uint8)  # Clip to valid pixel range and valid datatype for vision problems

# Compress image using SVD with different k values
compressed_img = compress_image_svd(img, 10) 

#Apply compress_image_svd (experiment with different values of k)

# Display Results
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(img, cmap="gray")
plt.title("Original Face Image")

plt.subplot(1, 3, 2)
plt.imshow(compressed_img, cmap="gray")
plt.title("Compressed Face Image")

plt.show()

NameError: name 'img' is not defined

# Step 3: Exposing the first limitation of SVD
Apply TruncatedSVD for Dimensionality Reduction (to test with non-linear data) and Compare with t-sne

**Note:** We will use in-built TruncatedSVD (**from sklearn.decomposition**) instead of using numpy to write SVD's maths

**We will generate a non-linear 3D dataset (Swiss Roll dataset), and apply TruncatedSVD on it and compare it with t-sne**

Complete the TODOs below

In [1]:
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

# Generate Swiss Roll dataset
n_samples = 1000
noise = 0.1
X, color = make_swiss_roll(n_samples, noise=noise)

# Apply SVD to reduce Swiss Roll from 3D to 2D
svd = TruncatedSVD(n_components=2) #TODO use TruncatedSVD() to initialize SVD by passing in number of components (n_components) as argument (this is k: number of top k values to retain)
X_svd = svd.fit_transform(X) #TODO fit and transform svd on the dataset X (use the fit_transform() function)

# Apply t-SNE to reduce Swiss Roll from 3D to 2D
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X)

# Plot Results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original 3D Swiss Roll
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.viridis)
ax.set_title("Original 3D Swiss Roll")

# SVD Projection
axes[1].scatter(X_svd[:, 0], X_svd[:, 1], c=color, cmap=plt.cm.viridis)
axes[1].set_title("SVD Projection to 2D")

# t-SNE Projection
axes[2].scatter(X_tsne[:, 0], X_tsne[:, 1], c=color, cmap=plt.cm.viridis)
axes[2].set_title("t-SNE Projection to 2D")

plt.show()

NameError: name 'plt' is not defined

#### **Observation**: 
t-sne preserves the curved structure but SVD fails to do so because it does not work with non-linear dataset, hence, it can not understand the relationship if it is non-linear. This exposes the first limitation of SVD (When data has non-linear relationship. )

# Step 4: Exposing the second limitation of SVD
Comparison of DCT vs SVD, and how SVD fails on text images (images with sharp edges and where there are abrupt changes in image intensity/brightness levels)

**There are no TODOs in this step, just run the cells below and observe the results!**

In [ ]:
image_path = "text.png"
img = load_image(image_path)
def max_svd_k(img):
    """Returns the maximum k value for SVD compression of an image."""
    m, n = img.shape  # Get height and width of the image
    return min(m, n)

plt.imshow(img, cmap="gray")
plt.title("Text Image (Dialogues from Breaking Bad :D)")
plt.show()

k_max = max_svd_k(img)
print(f"Maximum possible k for SVD: {k_max}")

#### **Maximum possible k for SVD in text image is: 814. We will retain the top 4 (top 0.5%) k (singular values) and observe the compressed image**. Run the below cell to apply SVD on text image

In [ ]:
def apply_svd_compression(img, k):
    """Compress the text image using Truncated SVD by keeping only k components"""
    svd = TruncatedSVD(n_components=k)
    
    # Fit SVD and transform the image
    U_reduced = svd.fit_transform(img)
    
    # Reconstruct the image from compressed representation
    compressed_img = svd.inverse_transform(U_reduced)
    
    return np.clip(compressed_img, 0, 255).astype(np.uint8)

compressed_img = apply_svd_compression(img, k=4)
    
plt.imshow(compressed_img, cmap="gray")
plt.title(f"Compressed text image using SVD (k=4) (top 0.5% k values)")
plt.axis("off")

plt.show()

#### Now we will apply DCT. We will retain the top 0.5% coefficient (frequency) values and observe the compressed image**. Run the below cell to apply DCT on text image

In [ ]:
from scipy.fftpack import dct, idct
def dct2(img):
    """Apply 2D Discrete Cosine Transform (DCT) to the image."""
    return dct(dct(img.T, norm='ortho').T, norm='ortho')

def idct2(dct_img):
    """Apply Inverse 2D Discrete Cosine Transform (IDCT) to reconstruct the image."""
    return idct(idct(dct_img.T, norm='ortho').T, norm='ortho')

def compress_image_dct(img, keep_fraction=0.1):
    """Compress the image by keeping only a fraction of the DCT coefficients."""
    dct_transformed = dct2(img)

    # Zero out small coefficients (Keep only top percentage of frequencies)
    keep_threshold = np.percentile(np.abs(dct_transformed), (1 - keep_fraction) * 100)
    dct_transformed[np.abs(dct_transformed) < keep_threshold] = 0

    # Reconstruct image using IDCT
    compressed_img = idct2(dct_transformed)

    return np.clip(compressed_img, 0, 255).astype(np.uint8)

# Load text image in grayscale
image_path = "text.png"  # Replace with actual file path
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Ensure image is loaded correctly
if img is None:
    raise FileNotFoundError(f"Error: Could not read the image file {image_path}. Please check the file path.")

# Apply DCT compression (keep 10% of the strongest coefficients)
compressed_img = compress_image_dct(img, keep_fraction=0.005)


plt.imshow(compressed_img, cmap="gray")
plt.title(f"Compressed Text Image using DCT (retained top 0.5% coefficients)")
plt.axis("off")

plt.show()

#### **Observation**:
We compared DCT and SVD on similar grounds by retaining the top 0.5% values in both the methods (top singular values for SVD and top coefficient values for DCT). We can clearly see how much better DCT performs.

This exposes the second limitation of SVD (When an image has sudden changes and sharp edges, like images of text)